In [1]:
import os
import numpy as np
import pandas as pd
import platform
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%qtconsole

In [3]:
if platform.system() == 'Linux':
    data_dir = '/home/alin/Data/Recruit_Holding'
else:
    data_dir = 'C:/Users/alin/Documents/Data/Recruit_Holding'

## Load Data

In [53]:
air_reserve = pd.read_csv(data_dir + '/air_reserve.csv.zip')

air_store_info = pd.read_csv(data_dir + '/air_store_info.csv.zip')

air_visit_hist = pd.read_csv(data_dir + '/air_visit_data.csv.zip')

date_info = pd.read_csv(data_dir + '/date_info.csv.zip')

hpg_reserve = pd.read_csv(data_dir + '/hpg_reserve.csv.zip')

hpg_store_info = pd.read_csv(data_dir + '/hpg_store_info.csv.zip')

sample_submission = pd.read_csv(data_dir + '/sample_submission.csv.zip')

store_id_relation = pd.read_csv(data_dir + '/store_id_relation.csv.zip')

## air_reserve
This dataframe contains reservations made in the air system. Note that the reserve_datetime indicates the time when the reservation was created, whereas the visit_datetime is the time in the future where the visit will occur.

- air_store_id - the restaurant's id in the air system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

In [32]:
air_reserve.head(3)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6


In [33]:
air_reserve.shape

(92378, 4)

In [34]:
air_reserve.dtypes

air_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors     int64
dtype: object

In [35]:
air_reserve.isnull().sum(axis=0)

air_store_id        0
visit_datetime      0
reserve_datetime    0
reserve_visitors    0
dtype: int64

### (1) separate date from time, (2) further pull out year, month and day and hour

In [62]:
def process_date_time(df, prefix):
    '''create new fields prefix_date, prefix_hour, prefix_year, prefix_month, prefix_day from prefix_datetime'''
    df[prefix + '_date'] = df.apply(lambda r: r[prefix + '_datetime'][:10], axis=1)    
    df[prefix + '_year'] = df.apply(lambda r: int(r[prefix + '_datetime'][:4]), axis=1)
    df[prefix + '_month'] = df.apply(lambda r: int(r[prefix + '_datetime'][5:7]), axis=1)
    df[prefix + '_day'] = df.apply(lambda r: int(r[prefix + '_datetime'][8:10]), axis=1)
    df[prefix + '_hour'] = df.apply(lambda r: int(r[prefix + '_datetime'][11:13]), axis=1)
    return df
    

In [73]:
air_reserve = process_date_time(air_reserve, 'visit')
air_reserve = process_date_time(air_reserve, 'reserve')

In [74]:
air_reserve.head(3) 

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,visit_year,visit_month,visit_day,visit_hour,reserve_date,reserve_year,reserve_month,reserve_day,reserve_hour
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,16
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,19
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,19


### append store info

In [87]:
air_reserve = air_reserve.merge(air_store_info, how='left')

### append weekday info

In [146]:
air_reserve = air_reserve.merge(date_info, how='left', left_on='visit_date', right_on='calendar_date')

In [152]:
air_reserve.drop(['visit_datetime', 'reserve_datetime', 'calendar_date'], axis=1, inplace=True)

In [153]:
air_reserve.head(3)

,air_store_id,reserve_visitors,visit_date,visit_year,visit_month,visit_day,visit_hour,reserve_date,reserve_year,reserve_month,reserve_day,reserve_hour,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg
0,air_877f79706adbfb06,1,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,16,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Friday,1
1,air_db4b38ebe7a7ceff,3,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,19,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,Friday,1
2,air_db4b38ebe7a7ceff,6,2016-01-01,2016,1,1,19,2016-01-01,2016,1,1,19,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,Friday,1


### No missing data

In [154]:
air_reserve.isnull().sum(axis=0)

air_store_id        0
reserve_visitors    0
visit_date          0
visit_year          0
visit_month         0
visit_day           0
visit_hour          0
reserve_date        0
reserve_year        0
reserve_month       0
reserve_day         0
reserve_hour        0
air_genre_name      0
air_area_name       0
latitude            0
longitude           0
day_of_week         0
holiday_flg         0
dtype: int64

### Create some counts as potential features

In [183]:
air_mean_reserve = air_reserve.groupby('air_store_id', as_index=False).sum()[['air_store_id','reserve_visitors']]

In [184]:
total_days = np.unique(air_reserve.visit_date).shape[0]

In [185]:
air_mean_reserve['avg_daily_air_reserve_visitors'] = air_mean_reserve['reserve_visitors'] / total_days

In [187]:
air_mean_reserve.drop('reserve_visitors', axis=1, inplace=True)

In [190]:
air_mean_reserve_wkday = air_reserve.groupby(['air_store_id', 'day_of_week'], as_index=False).sum()[['air_store_id', 'day_of_week',
                                                                                                    'reserve_visitors']]

In [225]:
wkday = air_reserve[['visit_date', 'day_of_week']].drop_duplicates()

In [228]:
wkday_cnt = wkday.day_of_week.value_counts()

In [231]:
air_mean_reserve_wkday['avg_air_reserve_visitors_wkday'] = air_mean_reserve_wkday.apply(lambda r: r['reserve_visitors'] / 
                                                                                       wkday_cnt[r['day_of_week']], axis=1)

In [234]:
DFS = {'air_reserve': air_reserve, 'air_mean_reserve': air_mean_reserve, 'air_mean_reserve_wkday': air_mean_reserve_wkday}

In [235]:
pickle.dump(DFS, open(data_dir + '/DFS', 'wb'))